In [ ]:
 #Verify the GPU 
import tensorflow as tf 
tf.test.gpu_device_name()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
##Open Cv version
import cv2
cv2.__version__                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [ ]:
##Installing the Tensorflow
!pip install tensorflow==2.2

In [ ]:
##Check Tensorflow version
import tensorflow
print(tensorflow.__version__)

In [ ]:
##Upload the Kaggle credentials File (Kaggle.json)
from google.colab import files
files.upload()

In [ ]:
## Execute the Kaggle Json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
## Download the German Traffic Sign Dataset(GTSRD)
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

In [ ]:
##Unzip The GTSRD Data
!unzip /content/gtsrb-german-traffic-sign.zip

In [ ]:
##Import Data
import pandas as pd
df = pd.read_csv('Train.csv')
print(df.head())

In [ ]:
##Create the Data Array and Array of Labels
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import random
data = []
labels = []
path = np.array(df.Path)
for i in path:
            image = cv2.imread(i)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((32, 32))
            data.append(np.array(size_image))
            labels.append(i.split('/')[1])
Cells=np.array(data)
labels=np.array(labels)
#print('Labels_shape',labels.shape)
#print('Cells_shape',Cells.shape)
plt.imshow(Cells[8025])  ##Example 8025
plt.axis('off')
plt.show()

In [ ]:
## Color Intensity of Image
import cv2 as cv
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv.calcHist([Cells[8025]],[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    plt.xlim([0,256])
plt.show()

In [ ]:
##Without Color Intensity V/s Pixels
plt.hist(Cells[8025].ravel(),256,[0,256])
hist,bins = np.histogram(Cells[8025].ravel(),256,[0,256])
plt.show()

In [ ]:
##Image Processing Functions 
import cv2
def grayscale(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return img

def equilizer(img):
  img = cv2.equalizeHist(img)
  return img

def preprocessing(img):
  img = grayscale(img)
  img = equilizer(img)
  img = img / 255
  return img

In [ ]:
##Plot Each Category of the Data
import matplotlib.pyplot as plt
sample = pd.read_csv('Meta.csv')
sample_data = sample.ClassId
num = []
fig=plt.figure(figsize=(10,10))	
plt.title("Traffic_Sign_Samples",color='black', pad = 45)
plt.axis(False)
for i in range(43):
	 sum = 0
	 ax = fig.add_subplot(7, 7, i+1)
	 ax.imshow(cv2.imread('/content/meta/{}.png'.format(i)))
	 ax.set_title(i, color ='black')
	 plt.xticks([])
	 plt.yticks([])
	 plt.tight_layout()
plt.savefig('Sample_data.png',bbox_inches='tight')
plt.show()

In [ ]:
##Ploting the Data Distribuction 
values = pd.value_counts(labels).keys().tolist()
counts = pd.value_counts(labels).tolist()
plt.figure(figsize=(20, 5))
plt.style.use('fivethirtyeight')
plt.bar(values, counts,color='#444444')
plt.title("Distribution of the Train Dataset",color='black')
plt.xlabel("Data Catagories",color='black')
plt.ylabel("Number of images",color='black')
plt.grid(True)
plt.savefig('Data_Distribuction.png',bbox_inches='tight')

In [ ]:
##Preprocessing the Data before going to Training 
import random
import cv2
import numpy as np
data_new_train = np.array(list(map(preprocessing,Cells)))
print(data_new_train.shape)
plt.imshow(data_new_train[8025],cmap = 'gray')
plt.axis('off')
#plt.savefig('Data.png',bbox_inches='tight')
plt.show()

In [ ]:
##Reshape the Data for the Requirements
data_new_train = data_new_train.reshape(39209,32,32,1)
#Spliting the images into train and validation sets
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(data_new_train, labels, test_size=0.2, random_state=42, shuffle=True)
print('train_data',train_x.shape)
print('Valid_data',val_x.shape)
print('train_data_label',train_y.shape)
print('Val_data_label',val_y.shape)

In [ ]:
#Using one hote encoding for the train and validation labels
from keras.utils import to_categorical
train_y = to_categorical(train_y, 43)
val_y = to_categorical(val_y, 43)
print('Train_data_label_shape',train_y.shape)
print('Validation_data_label_shape',val_y.shape)

#Model's (1-7)
choose only one model to predict the result

In [ ]:
##1.Base_line_Model (TSR-CNN)
from tensorflow.python.keras import Sequential
from contextlib import redirect_stdout
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.python.keras.regularizers import l2
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 1)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer = 'he_uniform'))
model.add(Dense(43, activation='softmax'))
# compile model
with open('TSR_CNN.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

In [ ]:
##2.TSR-CNN-1
##Base_Line_Model (TSR-CNN) + Weight_decay_model
from tensorflow.python.keras import Sequential
from contextlib import redirect_stdout
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.python.keras.regularizers import l2
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001), input_shape=(32, 32, 1)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.001)))
model.add(Dense(43, activation='softmax'))
# compile model
with open('TSR_CNN_1.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

In [ ]:
##3.TSR-CNN-2
##Base_line (TSR-CNN) + Dropout_model	
from tensorflow.python.keras import Sequential
from contextlib import redirect_stdout
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.python.keras.regularizers import l2
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 1)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
#model.add(Dropout(0.02))
model.add(Dense(43, activation='softmax'))
# compile model
with open('TSR_CNN_2.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

In [ ]:
##4.TSR-CNN-3
##Base_line_Model + Variance_Dropout
from tensorflow.python.keras import Sequential
from contextlib import redirect_stdout
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.python.keras.regularizers import l2
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 1)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(43, activation='softmax'))
# compile model
with open('TSR_CNN_3.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

In [ ]:
##5.TSR-CNN-4 
##Base_line_Model + Batch Normalization
from tensorflow.python.keras import Sequential
from contextlib import redirect_stdout
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.python.keras.regularizers import l2

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(43, activation='softmax'))

#compile model
with open('TSR_CNN_4.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

In [ ]:
##6.TSR-CNN-5 
##Base_line_Model + Dropout + Batch Normalization
from tensorflow.python.keras import Sequential
from contextlib import redirect_stdout
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.python.keras.regularizers import l2

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(43, activation='softmax'))

#compile model
with open('TSR_CNN_5.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

In [ ]:
##7.TSR-CNN-6 
##Base_line_Model + Variance_Dropout + Batch Normalization
from tensorflow.python.keras import Sequential
from contextlib import redirect_stdout
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.python.keras.regularizers import l2

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(43, activation='softmax'))

#compile model
with open('TSR_CNN_6.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

In [ ]:
# Clear logs files from previous training to get new results 
!rm -rf ./log/ 

In [ ]:
##Tenssore board
from tensorflow.keras.callbacks import TensorBoard
import datetime
import tensorflow as tf
log_dir = "log/" #atetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./log")
                                                  

In [ ]:
#Compilation of the model
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2 = 0.999, amsgrad=False)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Train the Model
epochs = 5
history = model.fit(train_x, train_y, batch_size = 64, epochs = epochs,shuffle= True, validation_data=(val_x, val_y),callbacks=[tensorboard_callback])

In [ ]:
## See the Loss and Accuracy in Tensorboard
#!pip install -U tensorboard
!tensorboard dev upload --logdir {"./log"}

In [ ]:
##Plotting the Graph of the Loss
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
#plt.subplot(211)
plt.figure(1)
plt.title('Loss')
plt.xlabel("Epochs",color='black')
plt.ylabel("Loss",color='black')
plt.plot(history.history['loss'],color = 'green',label = 'Train_Loss')
plt.plot(history.history['val_loss'],color = 'black',label = 'Val_Loss')
plt.legend()
plt.savefig('Loss_Baseline.png',bbox_inches='tight')

In [ ]:
## Accuracy Curve
plt.style.use('fivethirtyeight')
plt.title('Cross_Entropy_Accuracy')
plt.xlabel("Epochs",color='black')
plt.ylabel("Accuracy",color='black')
plt.plot(history.history['accuracy'],color = 'green',label = 'Train_Acc')
plt.plot(history.history['val_accuracy'],color = 'black',label = 'Val_Acc')
plt.legend()
plt.savefig('Accuracy_Baseline.png',bbox_inches='tight')

In [ ]:
##Pre-process test-data
import pandas as pd
import numpy as np
import cv2
from PIL import Image
#Predicting with the test data
test_y = pd.read_csv("Test.csv")
#labels=y_test['Path'].as_matrix()
test_labels = test_y['ClassId'].values
test_data = []
path_test = np.array(test_y.Path)
for i in path_test:
            image=cv2.imread(i)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((32, 32))
            test_data.append(np.array(size_image))
test_data = np.array(test_data)
test_data = np.array(list(map(preprocessing,test_data)))
test_data = np.array(test_data).astype(np.float32)
test_labels = np.array(test_labels)
##Shape of the Test Data
#print(test_data.shape)
print('Test_data_Shape',test_data.shape)
print('Test_label_shape',test_labels.shape)

In [ ]:
##Test_example data for prediction
plt.axis('off')
plt.imshow(test_data[12],cmap='gray')
print('Test_data_Label:', test_labels[12])

In [ ]:
##Prediction the Sample Data
#Convert as per requirements
test_data = test_data.reshape(12630,32,32,1)
##Predict the model
predict  = model.predict_classes(test_data)
print('Prediction label of above example (12) :', predict[12])
##auucracy score
from sklearn.metrics import accuracy_score
print('Total accuracy of the model', accuracy_score(test_labels,predict))

##Plot the prediction graph
#pred = model.predict(test_data.reshape(12630,32,32,1))
#y = list(pred[12])
#y_1 = [i*1000000 for i in y]
#x = [i for i in range(43)]
#print(y_1)
#print(x)
#plt.figure(figsize=(10, 5))
#plt.style.use('fivethirtyeight')
#plt.bar(x,y,color='#444444')
#plt.title("Distribution of the Train Dataset",color='black')
#plt.xlabel("Data Catagories",color='black')
#plt.ylabel("Number of images",color='black')
#plt.grid(True)
#plt.savefig('Data_Distribuction.png',bbox_inches='tight')"""
#from tensorflow.keras.utils import to_categorical
##Conver the Data As per Requirements
#test_data = test_data.reshape(12630,32,32,1)
#test_labels = to_categorical(test_labels, 43)
#print('test_data',test_data.shape)
#print('test_labels',test_labels.shape)
#score = model.evaluate(test_data,test_labels,verbose=1)
#print('Test_Score',score[0])
#print('Test_Accuracy',score[1])

In [ ]:
##Test the Image from the URL:
import requests
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
url = 'https://previews.123rf.com/images/pejo/pejo0907/pejo090700003/5155701-german-traffic-sign-no-205-give-way.jpg'#'https://previews.123rf.com/images/bwylezich/bwylezich1608/bwylezich160800375/64914157-german-road-sign-slippery-road.jpg'
r = requests.get(url,stream = True)
img = Image.open(r.raw)
plt.axis('off')
plt.imshow(img,cmap = 'gray' )

In [ ]:
## Predict the URL Data's
img = np.asarray(img)
img = cv2.resize(img,(32,32))
data = np.array(img)
data = data.reshape(1,32,32,3)
data = np.array(list(map(preprocessing,data)))
#plt.imshow(data[0],cmap='gray')
#plt.axis('off')
#plt.show()
##Model Prediction
#Resize as per requirments
img = data.reshape(1,32,32,1)
print('Testing image Class is :' + str(model.predict_classes(img)))

In [ ]:
##Confusion Matrix
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(test_labels, predict))
cnf_matrix = confusion_matrix(test_labels, predict)
print(cnf_matrix)
plt.savefig('l.png',bbox_inches='tight')

In [ ]:
# Plotting the confusion matrix
import itertools
names = [i for i in range(43)]
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion Matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(20,20))
    plt.imshow(cm, interpolation='nearest')
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],horizontalalignment="center",color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label',labelpad=20)
    plt.xlabel('Predicted label',labelpad=20)
    plt.grid(False)
plot_confusion_matrix(cnf_matrix, classes= names,title='Confusion matrix')
plt.savefig('Confusion_Matrix_baseline.png',bbox_inches='tight')

In [ ]:
##Summarize filter shapes
for layer in model.layers:
  # check for convolutional layer
  if 'conv' not in layer.name:
     continue
  # Get Filter Weights
  filters, biases = layer.get_weights()
  print(layer.name,filters.shape,biases.shape)

In [ ]:
## Visulization of the Some Filters 
import matplotlib.pyplot as pyplot

# retrieve weights from the first hidden layer
filters, biases = model.layers[0].get_weights()
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
# plot first few filters
n_filters, ix = 32,3  ## number of filter's and Layer number                    ## select the Layer here
fig=pyplot.figure(figsize=(25,25))	
for i in range(n_filters):
	# get the filter
	f = filters[:, :, :, i]
	# plot each channel separately
	for j in range(1):
		# specify subplot and turn of axis
		ax = fig.add_subplot(n_filters, 4, ix)
		ax.set_xticks([])
		ax.set_yticks([])
    
		# plot filter channel in grayscale
		pyplot.imshow(f[:, :, j], cmap='gray')
		ix += 1
# show the figure
plt.savefig(str(ix) + '_layer_weights.png',bbox_inches='tight')
pyplot.show()

In [ ]:
## Activation output of Test _example (12)
import theano
from keras import backend as K
from PIL import Image
##Conver the Data As per Requirements
test_image = test_data.reshape(12630,32,32)
test_image = test_image[12] ## To change
#print (test_image.shape)
plt.figure(figsize=(1,1))
plt.imshow(test_image)
plt.axis(False)
plt.savefig('test.png',bbox_inches='tight')
plt.show()
##Convert the image to array
#test_image = Image.fromarray(test_image)
##Expand the image as per Requirements
#test_image_f = np.expand_dims(test_image,axis=0)
#test_image_f = test_image_f.reshape(1,32,32,1)
#print(test_image_f.shape)##How the layer Visulize the image

In [ ]:
## Keras Sample-output visulization
from keras.models import Model
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(test_image.reshape(1,32,32,1))
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='gray')
            ax[row][col].axis('off')
            activation_index += 1
    plt.savefig('test.png',bbox_inches='tight')
## Display layer's and with filters 
## (filters, filters, Layer Number)
display_activation(activations, 8, 8, 3)       

In [ ]:
##To  Save  the Model in JSON Form
model_json = model.to_json()
with open('model.json','w') as json_file:
      json_file.write(model_json)
##To save the weight
model.save_weights('model_weight_json.h5')
print('Model has saved')

In [ ]:
##To Load json Model
from tensorflow.keras.models import model_from_json
json_file  = open('model.json')
load_model_json = json_file.read()
json_file.close()
##load
model_1 = model_from_json(load_model_json)
model_1.load_weights('model_weight_json.h5')
print('Model has loaded')

In [ ]:
##To save the model (YaMl)
model_yaml  = model.to_yaml()
with open('model.yaml','w') as yaml_file:
  yaml_file.write(model_yaml)
##To save the weight
model.save_weights('model_weight_yaml.h5')

print('Model has Saved')

In [ ]:
##To Load the model (YaMl)
from tensorflow.keras.models import model_from_yaml
yaml_file  = open('model.yaml')
load_model_yaml  = yaml_file.read()
yaml_file.close()

##To load the weights
model_2 = model_from_yaml(load_model_yaml)
model_2.load_weights('model_weight_yaml.h5')
print('Model has loaded')

In [ ]:
##To Save model with HDF5
model = model.save('final_model.h5')

In [ ]:
##To Load weights
from tensorflow.keras.models import load_model
model_3 = load_model('final_model.h5')